In [2]:
from statsbombpy import sb

df_camp = sb.competitions()
df_camp.head(5)

c:\Users\RodrigoPintoMesquita\Documents\GitHub\DR3_AT\.venv\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available
0,9,281,Germany,1. Bundesliga,male,False,False,2023/2024,2024-07-15T14:15:54.671676,2024-07-15T14:17:00.877356,2024-07-15T14:17:00.877356,2024-07-15T14:15:54.671676
1,9,27,Germany,1. Bundesliga,male,False,False,2015/2016,2024-05-19T11:11:14.192381,None,None,2024-05-19T11:11:14.192381
2,1267,107,Africa,African Cup of Nations,male,False,True,2023,2024-06-13T07:51:02.452825,None,None,2024-06-13T07:51:02.452825
3,16,4,Europe,Champions League,male,False,False,2018/2019,2024-06-12T07:44:38.559714,2021-06-13T16:17:31.694,None,2024-06-12T07:44:38.559714
4,16,1,Europe,Champions League,male,False,False,2017/2018,2024-02-13T02:35:28.134882,2021-06-13T16:17:31.694,None,2024-02-13T02:35:28.134882


In [3]:
df_matches = sb.matches(competition_id=16, season_id=4)
df_matches.head(3)

c:\Users\RodrigoPintoMesquita\Documents\GitHub\DR3_AT\.venv\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_status,...,last_updated_360,match_week,competition_stage,stadium,referee,home_managers,away_managers,data_version,shot_fidelity_version,xy_fidelity_version
0,22912,2019-06-01,21:00:00.000,Europe - Champions League,2018/2019,Tottenham Hotspur,Liverpool,0,2,available,...,2021-06-13T16:17:31.694,13,Final,\tEstádio Cívitas Metropolitano,Damir Skomina,Mauricio Roberto Pochettino Trossero,Jürgen Klopp,1.1.0,2,2


In [2]:
# Crie uma função simples que receba uma ID de partida e retorne os dados brutos dessa partida utilizando a API.
from statsbombpy import sb

df_events = sb.events(match_id=3795108)
df_events

c:\Users\RodrigoPintoMesquita\Documents\GitHub\DR3_AT\.venv\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


,50_50,ball_receipt_outcome,ball_recovery_offensive,ball_recovery_recovery_failure,block_deflection,block_offensive,carry_end_location,clearance_aerial_won,clearance_body_part,clearance_head,...,substitution_outcome,substitution_outcome_id,substitution_replacement,substitution_replacement_id,tactics,team,team_id,timestamp,type,under_pressure
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"{'formation': 4231, 'lineup': [{'player': {'id...",Switzerland,773,00:00:00.000,Starting XI,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"{'formation': 433, 'lineup': [{'player': {'id'...",Spain,772,00:00:00.000,Starting XI,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Spain,772,00:00:00.000,Half Start,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Switzerland,773,00:00:00.000,Half Start,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Spain,772,00:00:00.000,Half Start,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4985,"{'outcome': {'id': 1, 'name': 'Lost'}}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Switzerland,773,00:04:36.095,50/50,True
4986,"{'outcome': {'id': 4, 'name': 'Won'}}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Spain,772,00:04:36.095,50/50,True
4987,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Spain,772,00:22:39.919,Error,NaN
4988,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Spain,772,00:28:49.144,Shield,True


In [21]:
df_events.shot_type.unique()

array([nan, 'Open Play', 'Penalty'], dtype=object)

In [ ]:
# EXERCICIO 4
# Desenvolva uma funcionalidade que permita criar um perfil detalhado de um jogador específico da partida, contendo:
#   Nome do jogador.
#   Estatísticas como número de passes, finalizações, desarmes e minutos jogados.
#   Retorne os dados de forma clara e organizada (e.g., JSON ou tabela).

### ACHO QUE VOU FAZER COM SCRIPT E NÃO COM MODELO (VER  O QUE Q O FERNANDO FALOU NA AULA)

In [ ]:
# EXERCICIO 5
# Desenvolvimento de uma API com FastAPI:
#       /match_summary: Retorna a sumarização de uma partida.
#       /player_profile: Retorna o perfil detalhado de um jogador.
# Utilize Pydantic para validar as entradas e saídas.

In [ ]:
# EXERCICIO 6
# Desenvolva uma funcionalidade para criar narrativas baseadas nos eventos da partida.
# Permita ao usuário escolher entre estilos de narração, como:
#       Formal (técnico e objetivo), Humorístico (descontraído e criativo), Técnico (análise detalhada dos eventos)

In [ ]:
#EXERCICIO 7
# Desenvolva uma interface simples que permita ao usuário:
#      Selecionar uma partida por ID.
#      Exibir os eventos principais e o perfil de um jogador.

In [ ]:
#EXERCICIO 8
#Implemente um agente LangChain para realizar:
#   Consulta de eventos específicos da partida.
#   Geração de comparações entre dois jogadores.
#   Habilite o agente a responder perguntas como:
#       "Quem deu mais passes na partida?"
#       "Qual jogador teve mais finalizações no primeiro tempo?"

######### ACHO QUE AQUI EU TENHO CRIAR UMA TOOL PERSONALIZADA QUE FAÇA O CALCULO E RETORNE PARA O MODELO  (ASSISTIR A AULA EM QUE O FERNANDO ENSINA A CRIAR UMA TOOL PERSONALIZADA) 
######### LINK ENSINANDO A CRIAR A FERRAMENTA PERSONALIZADA https://python.langchain.com/docs/concepts/tools/



#Se a pergunta for sobre um jogador ele deve realizar um filtro no DataFrame de eventos para obter os eventos do jogador já agregados.
    #Agregar por jogador, mantendo o tempo e o time. 
      #O Agente escolhe uma tool desenvolvida para calcular as estatísticas de um jogador. (Variável: Nome do Jogador)
        #A tool recebe o nome do jogador, filtra o dataframe de eventos e retorna o dataframe agregado.
    #A tool para comparar dois jogadores faz a mesma coisa, porém filtrando dois jogadores. (Variáveis: Nome do Jogador 1 e Nome do Jogador 2)
    #A tool para eventos específicos da partida faz um filtro no dataframe de eventos. (Variável: Nome do Evento)
        #A tool recebe o nome do evento, filtra o dataframe de eventos e retorna agregado por time, jogador, evento, tempo do jogo.

In [5]:
df_events = sb.events(match_id = 22912)

colunas_desejadas = set(['second','period','team','player','pass_goal_assist','bad_behaviour_card','shot_type','shot_outcome'])
colunas_no_df = set(df_events.columns)
filtro_colunas = list(colunas_desejadas & colunas_no_df)

# Filtrando o DataFrame para incluir apenas as colunas que estão presentes, pois algumas colunas não aparecem para todas as partidas
X = df_events[filtro_colunas]

melt_values_desejado = set(['pass_goal_assist','bad_behaviour_card','shot_outcome'])
melt_values = list(melt_values_desejado & colunas_no_df)

X = X.melt(id_vars=['second','period','team','player','shot_type'], value_vars=melt_values)
X = X[X.value.notnull()]


#Tratar os tipos de evento
for i, row in X.iterrows():
    if row['variable'] == 'bad_behaviour_card' and row['value'] == 'Yellow Card':
        X.at[i,'play'] = 'YELLOW CARD'
    elif row['variable'] == 'bad_behaviour_card' and row['value'] == 'Red Card':
        X.at[i,'play'] = 'RED CARD'
    elif row['value'] == 'Blocked':
        X.at[i,'play'] = 'SHOT BLOCKED BY THE DEFENSE'
    elif row['value'] == 'Saved':
        X.at[i,'play'] = 'SHOT SAVED BY THE GOALKEEPER'
    elif row['shot_type'] == 'Penalty' and row['value'] == 'Goal':
        X.at[i,'play'] = 'PENALTY GOAL SCORED'
    elif row['shot_type'] == 'Open Play' and row['value'] == 'Goal':
        X.at[i,'play'] = 'Successful SHOT, Goal Scored'
    elif row['variable'] == 'shot_outcome':
        X.at[i,'play'] = 'UNSUCCESSFUL SHOT'
    elif row['variable'] == 'pass_goal_assist' and row['value'] == True:
        X.at[i,'play'] = 'ASSIST PASS'
    else:
        X.at[i,'play'] = 'EVENTO NAO MAPEADO'
    

period_map = {
    1: 'first half',
    2: 'second half'
}

X['match_period'] = X['period'].apply(lambda k: period_map.get(k, 'Nao Mapeado'))

#Ordem cronológica
X = X.sort_values(['period','second']).reset_index()

# Ordenar colunas
df_main_events = X[['match_period','team','player','play']]


df_main_events


c:\Users\RodrigoPintoMesquita\Documents\GitHub\DR3_AT\.venv\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


,match_period,team,player,play
0,first half,Tottenham Hotspur,Moussa Sissoko,UNSUCCESSFUL SHOT
1,first half,Liverpool,Jordan Brian Henderson,SHOT BLOCKED BY THE DEFENSE
2,first half,Tottenham Hotspur,Christian Dannemann Eriksen,UNSUCCESSFUL SHOT
3,first half,Liverpool,Mohamed Salah,SHOT BLOCKED BY THE DEFENSE
4,first half,Liverpool,Mohamed Salah,UNSUCCESSFUL SHOT
5,first half,Liverpool,Mohamed Salah,PENALTY GOAL SCORED
6,first half,Liverpool,Trent Alexander-Arnold,UNSUCCESSFUL SHOT
7,first half,Liverpool,Andrew Robertson,SHOT SAVED BY THE GOALKEEPER
8,first half,Liverpool,Mohamed Salah,SHOT BLOCKED BY THE DEFENSE
9,first half,Liverpool,Mohamed Salah,UNSUCCESSFUL SHOT


In [ ]:
# EXERCICIO 9
# Melhore a interface Streamlit, adicionando:
#   Filtros para explorar eventos específicos (e.g., apenas gols ou cartões).
#   Visualizações gráficas para estatísticas dos jogadores (e.g., gráficos de barras, tabelas interativas).
# Comparação direta entre dois jogadores selecionados.

############ PEGAR DO TP3, JÁ TEM ISSO PRONTO

In [7]:
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from dotenv import load_dotenv
import google.generativeai as genai
import pandas as pd
from langchain.agents import Tool
from langchain.tools import tool
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent


load_dotenv()

google_api_key = os.getenv('GOOGLE_API_KEY')

def half_summarizer(df: pd.DataFrame, half: str) -> tuple:
    ''' 
    Cria um agente para analisar um tempo da partida.

    Args:
    df: DataFrame com os eventos da partida.
    half: String com o tempo da partida a ser analisado, esperado os valores "firt half" ou "second half".
    '''


    prompt = f"""
        As a football commentator, summarize in the main events of the match. Follow the instructions bellow:
            1. The data received represent only the events of {half}
            2. It's mandatory to comment on all the goals scored and pennaltys.
            3. Pay attention to the chronological order in the index column, and emphasize the teams and players who made each play. 
            4. End the comment by stating the final score of the half, for example: Team A 1 - 2 Team B.
    """

    llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=google_api_key)

    df = df[df['match_period'] == half]
    agent = create_pandas_dataframe_agent(llm, df, verbose=True, allow_dangerous_code = True)

    return agent, prompt


def match_summarizer(df: pd.DataFrame) -> str:
    '''Invoca Agentes para analisar cada tempo da partida de forma separada e depois gera um resumo final

    Args:
    df: DataFrame com os eventos principais da partida, já filtrado e com tratamento dos labels dos eventos.
    '''

    agent,prompt = half_summarizer(df, 'first half')
    first_half_analysis = agent.invoke(prompt)
    first_half_analysis = first_half_analysis['output']

    agent,prompt = half_summarizer(df, 'second half')
    second_half_analysis = agent.invoke(prompt)
    second_half_analysis = second_half_analysis['output']

    match_analyses = "ANALYSES FIRST HALF: \n" + first_half_analysis + "\nANALYSES SECOND HALF: \n" +second_half_analysis + "\n"

    return match_analyses

def match_stats(df: pd.DataFrame, user_input: str) -> str:
    
    prompt = f"""
        Analyze the dataframe to answer questions about soccer matches. 
        You need to answer the question {user_input}.
        Firts, check if the event, team name, or player name is present in the dataset, creating a list of unique values of related column, if not, answer with the available options in the dataset.
        Please provide a complete and detailed response. For example, if the question is 'Which player scored the most goals in the match?', I would like you to respond with something like 'The player who scored the most goals in the match was Ronaldo Gaucho.

        The provided Dataframe has the following fields:
            -match_period: Indication of the period of the game in which the play was made
            -team: Team that executed the play
            -player: Name of the player who executed the play
            -play: Specification of the executed play, also called an event.
        
        Description of each value in the column "play":

            -Yellow Card: Player penalized with a yellow card
            -Red Card: Player penalized with a red card
            -Shot Blocked: Shots on goal that were blocked or saved
            -Goal Scored: Shots on goal that were successful during the match
            -Penalty Goal Scored: Shots on goal that were successful through penalty kicks
            -Assist Pass: Pass of the ball to another player, who scored a goal
        
        Examples of Questions and Instructions:
            -Comparisons between two players: 
                Example: Compare the Player A and Player B.
                Instruction: Filter the players' names in the dataset and aggregate to count the events for each. Analyse the results and give a report.
            -Questions about a particular type of event in the match. 
                Example: How many yellow cards were given in the first half?
                Instruction: Filter the event type in the dataset and aggregate to count the total. Consider the match time and the teams.
            -Questions about rankings. 
                Examples: Which player received the most yellow cards? Wich player did the most assists?
                Instruction: Filter the dataset, aggregate, and sort the values from highest to lowest
        """
    
    df = df.groupby(['match_period','team','player','play']).size().reset_index(name='count')

    llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=google_api_key)

    agent = create_pandas_dataframe_agent(llm, df, verbose=True, allow_dangerous_code = True, number_of_head_rows=200)
    response = agent.invoke(prompt)
    
    return response['output']


def create_tools(df, user_input):
    return [
        Tool(
            #O nome do prompt não condiz com o conteúdo, pois não estou conseguindo trocar o nome do meu projeto no LangSmith.
            name="musicindustrysearch/match_summarizer",
            func=lambda x="": match_summarizer(df), 
            description="Creates narrative summaries about the match."
        ),
        Tool(
            name="match_stats", 
            func=lambda  x="":match_stats(df, user_input),  
            description="Provides statistical data about the match, teams, and players. Example: Number of saves, goals, shots on goal, etc."
        )
    ]
     



def invoke_agent(df, user_input, llm_tone):
    tools = create_tools(df, user_input)

    agent = create_react_agent(
        llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash",temperature=0.7), 
        tools = tools, 
        prompt = hub.pull("hwchase17/react")
        )
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, max_turns=2)
    
    return agent_executor



def main_llm(user_input, df, llm_tone) -> str:

    map_llm_tone = {
    "Formal" : "formal (technical and objective)",
    "Excited" :  "excited (vibrant, celebratory)",
    "Humorous" : "humorous (relaxed and creative)",
    "Technical" : "technical (detailed analysis of events)"
    }

    llm_tone = map_llm_tone.get(llm_tone, "Formal") 

    agent = invoke_agent(df,user_input, llm_tone)
    response = agent({"input": user_input})
    return response['output']

user_input = "Which player scored the most goals in the match?"
llm_tone = "Formal"
main_llm(user_input, df_main_events, llm_tone)

C:\Users\RodrigoPintoMesquita\AppData\Local\Temp\ipykernel_15124\923374285.py:151: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent({"input": user_input})




> Entering new AgentExecutor chain...
Thought: To answer this question, I need the match statistics, specifically the goals scored by each player.
Action: match_stats
Action Input: ''

> Entering new AgentExecutor chain...
Thought: I need to find the players who scored goals and count their goals.  The relevant events are 'PENALTY GOAL SCORED' and 'Successful SHOT, Goal Scored'. I'll filter the dataframe to include only those events, group by player, and sum the counts. Then I'll find the player with the maximum goal count.

Action: python_repl_ast
Action Input:
```python
import pandas as pd
data = {'match_period': ['first half', 'first half', 'first half', 'first half', 'first half', 'first half', 'first half', 'first half', 'second half', 'second half', 'second half', 'second half', 'second half', 'second half', 'second half', 'second half', 'second half', 'second half', 'second half', 'second half', 'second half', 'second half', 'second half', 'second half', 'second half', 'second

{'input': 'Which player scored the most goals in the match?',
 'output': 'Mohamed Salah'}

In [8]:
df_matches

,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_status,match_status_360,last_updated,last_updated_360,match_week,competition_stage,stadium,referee,home_managers,away_managers,data_version,shot_fidelity_version,xy_fidelity_version
0,22912,2019-06-01,21:00:00.000,Europe - Champions League,2018/2019,Tottenham Hotspur,Liverpool,0,2,available,scheduled,2022-11-28T14:45:35.628134,2021-06-13T16:17:31.694,13,Final,\tEstádio Cívitas Metropolitano,Damir Skomina,Mauricio Roberto Pochettino Trossero,Jürgen Klopp,1.1.0,2,2


In [39]:
def get_lineup(match_id):
    home_team = df_matches_filt['home_team'][0]
    away_team = df_matches_filt['away_team'][0]

    df_home = sb.lineups(match_id=df_matches_filt['match_id'][0])[home_team]
    df_away = sb.lineups(match_id=df_matches_filt['match_id'][0])[away_team]

    df_home['team'] = home_team
    df_away['team'] = away_team


    df_lineup = pd.concat([df_home, df_away])
    df_lineup = df_lineup[['team','player_name']]

    lineup_dict = {
        home_team: df_lineup[df_lineup['team'] == home_team]['player_name'].tolist(),
        away_team: df_lineup[df_lineup['team'] == away_team]['player_name'].tolist()
    }

    return lineup_dict 

c:\Users\RodrigoPintoMesquita\Documents\GitHub\DR3_AT\.venv\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


,team,player_name
0,Tottenham Hotspur,Christian Dannemann Eriksen
1,Tottenham Hotspur,Kyle Walker-Peters


In [48]:
import sys
sys.path.append(r'C:\Users\RodrigoPintoMesquita\Documents\GitHub\DR3_AT')

from statsbombpy import sb

sb.competitions()

c:\Users\RodrigoPintoMesquita\Documents\GitHub\DR3_AT\.venv\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available
0,9,281,Germany,1. Bundesliga,male,False,False,2023/2024,2024-07-15T14:15:54.671676,2024-07-15T14:17:00.877356,2024-07-15T14:17:00.877356,2024-07-15T14:15:54.671676
1,9,27,Germany,1. Bundesliga,male,False,False,2015/2016,2024-05-19T11:11:14.192381,None,None,2024-05-19T11:11:14.192381
2,1267,107,Africa,African Cup of Nations,male,False,True,2023,2024-06-13T07:51:02.452825,None,None,2024-06-13T07:51:02.452825
3,16,4,Europe,Champions League,male,False,False,2018/2019,2024-06-12T07:44:38.559714,2021-06-13T16:17:31.694,None,2024-06-12T07:44:38.559714
4,16,1,Europe,Champions League,male,False,False,2017/2018,2024-02-13T02:35:28.134882,2021-06-13T16:17:31.694,None,2024-02-13T02:35:28.134882
...,...,...,...,...,...,...,...,...,...,...,...,...
69,55,43,Europe,UEFA Euro,male,False,True,2020,2024-04-16T12:44:40.558402,2024-04-16T12:47:18.505110,2024-04-16T12:47:18.505110,2024-04-16T12:44:40.558402
70,35,75,Europe,UEFA Europa League,male,False,False,1988/1989,2024-02-12T14:45:05.702250,2021-06-13T16:17:31.694,None,2024-02-12T14:45:05.702250
71,53,106,Europe,UEFA Women's Euro,female,False,True,2022,2024-02-13T13:27:17.178263,2024-02-13T13:30:52.820588,2024-02-13T13:30:52.820588,2024-02-13T13:27:17.178263
72,72,107,International,Women's World Cup,female,False,True,2023,2024-07-14T16:59:48.469596,2024-07-14T17:01:38.887279,2024-07-14T17:01:38.887279,2024-07-14T16:59:48.469596


In [2]:
from statsbombpy import sb

df = sb.competitions()
df = df[['competition_name', 'season_name', 'competition_id', 'season_id']]
dict = df.to_dict(orient='records') 
dict

c:\Users\RodrigoPintoMesquita\Documents\GitHub\DR3_AT\.venv\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


[{'competition_name': '1. Bundesliga',
  'season_name': '2023/2024',
  'competition_id': 9,
  'season_id': 281},
 {'competition_name': '1. Bundesliga',
  'season_name': '2015/2016',
  'competition_id': 9,
  'season_id': 27},
 {'competition_name': 'African Cup of Nations',
  'season_name': '2023',
  'competition_id': 1267,
  'season_id': 107},
 {'competition_name': 'Champions League',
  'season_name': '2018/2019',
  'competition_id': 16,
  'season_id': 4},
 {'competition_name': 'Champions League',
  'season_name': '2017/2018',
  'competition_id': 16,
  'season_id': 1},
 {'competition_name': 'Champions League',
  'season_name': '2016/2017',
  'competition_id': 16,
  'season_id': 2},
 {'competition_name': 'Champions League',
  'season_name': '2015/2016',
  'competition_id': 16,
  'season_id': 27},
 {'competition_name': 'Champions League',
  'season_name': '2014/2015',
  'competition_id': 16,
  'season_id': 26},
 {'competition_name': 'Champions League',
  'season_name': '2013/2014',
  'com

In [ ]:
df_events = sb.events(match_id = 22912)

c:\Users\RodrigoPintoMesquita\Documents\GitHub\DR3_AT\.venv\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


In [29]:
colunas_desejadas = set(['team','type', 'period', 'player', 'shot_outcome', 'pass_outcome', 'pass_goal_assist','bad_behaviour_card'])
colunas_no_df = set(df_events.columns)
filtro_colunas = list(colunas_desejadas & colunas_no_df)

df_events = df_events[filtro_colunas]

jogadas_desejadas = ['Block', 'Dribble', 'Foul Won', 'Shot', 'Pass']
df_events = df_events[df_events['type'].isin(jogadas_desejadas)]
df_events

,pass_goal_assist,shot_outcome,period,pass_outcome,type,player,team
6,NaN,NaN,1,NaN,Pass,Jordan Brian Henderson,Liverpool
7,NaN,NaN,1,Incomplete,Pass,Joël Andre Job Matip,Liverpool
8,NaN,NaN,1,Incomplete,Pass,Kieran Trippier,Tottenham Hotspur
9,NaN,NaN,1,Incomplete,Pass,Fábio Henrique Tavares,Liverpool
10,NaN,NaN,1,Incomplete,Pass,Moussa Sissoko,Tottenham Hotspur
...,...,...,...,...,...,...,...
3134,NaN,NaN,2,NaN,Block,Heung-Min Son,Tottenham Hotspur
3135,NaN,NaN,2,NaN,Block,Eric Dier,Tottenham Hotspur
3136,NaN,NaN,2,NaN,Block,James Philip Milner,Liverpool
3137,NaN,NaN,2,NaN,Block,Eric Dier,Tottenham Hotspur


In [31]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

colunas_desejadas = set(['team','type', 'period', 'player', 'shot_outcome', 'pass_outcome', 'pass_goal_assist','bad_behaviour_card'])
colunas_no_df = set(df_events.columns)
filtro_colunas = list(colunas_desejadas & colunas_no_df)

df_events = df_events[filtro_colunas]

jogadas_desejadas = ['Block', 'Dribble', 'Foul Won', 'Shot', 'Pass']
df_events = df_events[df_events['type'].isin(jogadas_desejadas)]

df_goals = df_events[df_events['shot_outcome'] == 'Goal'].copy()
df_goals['outcome'] = 'Goal'
df_assistances = df_events[df_events['pass_goal_assist'] == True].copy()
df_assistances['outcome'] = 'Goal Assistance'

for i, row in df_events.iterrows():
    if row['type'] == 'Shot':
        df_events.at[i, 'outcome'] = 'Total Shots'
    elif row['type'] == 'Pass':
        df_events.at[i, 'outcome'] = 'Total Passes'
    elif row['type'] == 'bad_behaviour_card':
        df_events.at[i, 'outcome'] = row['bad_behaviour_card']
    else:
        df_events.at[i, 'outcome'] = row['type']


df = pd.concat([df_goals, df_assistances, df_events])
df = df[['team','period','player', 'outcome']]




period_map = {
    1: 'first half',
    2: 'second half',
    3: 'first extra time',
    4: 'second extra time',
    5: 'penalty shootout'
}

df['period'] = df['period'].apply(lambda k: period_map.get(k, 'Nao Mapeado'))


def dash_events():
    period = st.selectbox('Select the Period', df['player'].unique())

    if period != 'hole match':
        df = df[df['period'] == period]

    df = df.groupby(['outcome']).size().reset_index(name='count')
    df = df[df['outcome'].notnull()].sort_values('count', ascending=False)


    sns.set_theme(style="whitegrid")
    plt.figure(figsize=(12,6))
    ax = sns.barplot(x="outcome", y="count", data=df)
    plt.xticks(rotation=45)
    plt.title('Eventos da Partida')
    plt.ylabel('Quantidade')
    plt.xlabel('Evento')

    st.seaborn(ax)

,team,period,player,outcome
2796,Liverpool,first half,Mohamed Salah,Goal
2822,Liverpool,second half,Divock Okoth Origi,Goal
842,Liverpool,second half,Joël Andre Job Matip,Goal Assistance
6,Liverpool,first half,Jordan Brian Henderson,Total Passes
7,Liverpool,first half,Joël Andre Job Matip,Total Passes
...,...,...,...,...
3134,Tottenham Hotspur,second half,Heung-Min Son,Block
3135,Tottenham Hotspur,second half,Eric Dier,Block
3136,Liverpool,second half,James Philip Milner,Block
3137,Tottenham Hotspur,second half,Eric Dier,Block
